In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow

  Using cached keras-3.3.3-py3-none-any.whl (1.1 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [ ]:
#library
import numpy as np
import sys
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

Trong bài toán này, dữ liệu là cuốn truyện "Alice ở xứ sở thần tiên" được cung cấp dưới dạng ASCII, bài toán sẽ học cách sinh ra một kí tự dựa vào một chuỗi kí tự phía trước của nó. Từ việc sinh ra 1 kí tự , kết hợp ta sinh ra một đoạn văn bản từ những kí tự ban đầu.

In [ ]:
#load_data
filename = "/content/drive/MyDrive/Alice_Text_LSTM/wonderland.txt"
#chuyển hết các dạng kí tự về kí tự viết thường
raw_text = open(filename, "r",encoding = "utf-8 ").read()
raw_text = raw_text.lower()

In [ ]:
chars = sorted(list(set(raw_text))) #tạo ra 1 list kí tự
char_to_int = dict((c,i) for i ,c in enumerate(chars)) #chuyển kí tự về số nguyên
int_to_char = dict((i,c) for  i ,c in enumerate(chars))

In [ ]:
#tóm tắt lại việc xử lí data đầu vào
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total characters:" , n_chars)
print("Total Vobab:", n_vocab)

Total characters: 144512
Total Vobab: 45


Có thể thấy rằng chúng ta có gần 150,000 kí tự khi đã chuyển đổi về kí tự viết thường. chỉ có 45 kí tự phân biệt, nhiều hơn so với bản chuẩn chữ cái tiếng anh chỉ có 26 chữ cái.

In [ ]:
#chia cuốn sách thành các chuỗi kí tự có độ dài 100
seq_length = 100 #đầu vào của mạng sẽ là 100 kí tự
dataX = []
dataY = []
for i in range(0,n_chars-seq_length,1):
    seq_in = raw_text[i:i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns :" ,n_patterns)

Total Patterns : 144412


In [ ]:
X = np.reshape(dataX ,(n_patterns , seq_length,1))
#normalize
X = X /float(n_vocab)
#one_hot_encoding
y = to_categorical(dataY)

In [ ]:
!pip install keras==2.4.3

In [ ]:
# tạo nên 1 mô hình LSTM
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# tạo nên các file checkpoint lưu việc training
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.compile(loss = "categorical_crossentropy",optimizer = "adam")
model.fit(X,y,epochs = 20,batch_size = 128 , callbacks = callbacks_list)

In [ ]:
# load lại file trọng số đã lưu của mạng
filename = "weights-improvement-47-1.2219-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# lựa chọn seed ngẫu nhiên để tạo test
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")